In [1]:
# 1. Library 호출
from pykrx import stock                     # 주가정보 호출 라이브러리
from scipy.stats.mstats import winsorize    # 전처리 : 이상치처리
import scipy.stats as ss                    # 전처리 : 정규화
import time                                 # 호출 1초 지연 -> API 차단 방지
import plotly.express as px                 # 시각화
import plotly.graph_objects as go           # 시각화
from plotly.subplots import make_subplots 
import matplotlib.pyplot as plt  # 시각화
import seaborn as sns

import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
tickers_23 = pd.DataFrame(stock.get_market_ticker_list("20230601"),columns=['23'])
tickers_24 = pd.DataFrame(stock.get_market_ticker_list("20240603"),columns=['24'])
tic = pd.merge(tickers_23,tickers_24,left_on='23',right_on='24',how='inner')
tickers_final = tic['24']
tickers_final =pd.DataFrame({'code' :tickers_final})

In [3]:
def calculate_kospi_mdd(start_date, end_date):
    # 코스피 지수 데이터를 불러오기
    df = stock.get_index_ohlcv_by_date(start_date, end_date, "1001")

    # 종가 기준으로 누적 최고점 계산
    df['Cumulative_Max'] = df['종가'].cummax()

    # 각 날짜별 낙폭 계산
    df['Drawdown'] = df['종가'] / df['Cumulative_Max'] - 1

    # MDD 계산 (낙폭의 최소값)
    mdd = df['Drawdown'].min()

    return mdd, df

# 예시: 코스피 지수의 MDD 계산
start_date = "20230601"
end_date = "20240603"

mdd, df = calculate_kospi_mdd(start_date, end_date)

print(f"KOSPI MDD (Maximum Drawdown): {mdd * 100:.2f}%")

KOSPI MDD (Maximum Drawdown): -14.59%


In [4]:
def calculate_mdd(stock_code, start_date, end_date):
    # 주식 데이터 불러오기
    df = stock.get_market_ohlcv_by_date(start_date, end_date, stock_code)
    
    # 종가 기준으로 누적 최고점 계산
    df['Cumulative_Max'] = df['종가'].cummax()

    # 각 날짜별 낙폭 계산
    df['Drawdown'] = df['종가'] / df['Cumulative_Max'] - 1

    # MDD 계산 (낙폭의 최소값)
    mdd = df['Drawdown'].min()

    return mdd#, df


start_date = "20230601"
end_date = "20240603"

mdd_df = pd.DataFrame(columns=['code','MDD'])
for ticker in tickers_final['code']:
    mdd = calculate_mdd(ticker, start_date, end_date) #, df 
    new_row = pd.DataFrame({'code': [ticker], 'MDD': [mdd]})
    mdd_df = pd.concat([mdd_df, new_row], ignore_index=True)

mdd_df.set_index('code',inplace=True)


In [5]:
def calculate_annual_mdd(stock_code, start_date, end_date):
    # 주가 데이터 불러오기
    df = stock.get_market_ohlcv_by_date(start_date, end_date, stock_code)
    df = df.reset_index()

    # 연도 컬럼 추가
    df['Year'] = df['날짜'].dt.year

    # 연도별로 MDD 계산
    annual_mdds = []

    for year, group in df.groupby('Year'):
        group['Cumulative_Max'] = group['종가'].cummax()
        group['Drawdown'] = group['종가'] / group['Cumulative_Max'] - 1
        mdd = group['Drawdown'].min()
        annual_mdds.append({'Year': year, 'MDD': mdd * 100})

    # 결과를 데이터프레임으로 변환
    mdd_df = pd.DataFrame(annual_mdds)
    
    return mdd_df

# 예시: 삼성전자(005930)의 연간 MDD 계산
stock_code = "005930"  # 삼성전자 종목 코드
start_date = "20190101"
end_date = "20231231"

mdd_df = calculate_annual_mdd(stock_code, start_date, end_date)

In [6]:
def calculate_market_annual_mdd(start_date, end_date):
    # 코스피 지수 데이터를 불러오기
    df = stock.get_index_ohlcv_by_date(start_date, end_date, "1001")  # 1001은 KOSPI 지수 코드
    df = df.reset_index()

    # 연도 컬럼 추가
    df['Year'] = df['날짜'].dt.year

    # 연도별로 MDD 계산
    annual_mdds = []

    for year, group in df.groupby('Year'):
        group['Cumulative_Max'] = group['종가'].cummax()
        group['Drawdown'] = group['종가'] / group['Cumulative_Max'] - 1
        mdd = group['Drawdown'].min()
        annual_mdds.append({'Year': year, 'MDD': mdd * 100})

    # 결과를 데이터프레임으로 변환
    mdd_df = pd.DataFrame(annual_mdds)
    
    return mdd_df

# 예시: 코스피 지수의 연간 MDD 계산
start_date = "20190101"
end_date = "20231231"

mdd_df = calculate_market_annual_mdd(start_date, end_date)

# 결과 출력
print(mdd_df)

   Year        MDD
0  2019 -15.072288
1  2020 -35.708898
2  2021 -14.105004
3  2022 -27.891705
4  2023 -14.588294
